In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot as plt


(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

class Neural_network:
    np.random.seed(10)
    def __init__(self,x_train,y_train,input_dim,hidden_layers_size,hidden_layers,output_dim,batch_size=30,epochs=10,activation_func="relu"
           ,learning_rate=6e-3 ,decay_rate=0.9,beta=0.9,beta1=0.9,beta2=0.99,optimizer="adam",weight_init="xavier"):

        self.x_train,self.x_cv,self.y_train,self.y_cv = train_test_split(x_train, y_train, test_size=0.10, random_state=100,stratify=y_train)

        np.random.seed(10)
        self.input_dim = input_dim
        self.hidden_layers = hidden_layers
        self.hidden_layers_size = hidden_layers_size
        self.output_dim = output_dim

        self.batch = batch_size
        self.epochs = epochs
        self.activation_func = activation_func
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.optimizer = optimizer
        self.weight_init = weight_init
        self.beta = beta
        self.beta1 = beta1
        self.beta2 = beta2


        self.layers = [self.input_dim] + self.hidden_layers*[self.hidden_layers_size] + [self.output_dim]

        layers = self.layers.copy()
        self.weights = []
        self.biases = []
        self.activations = []
        self.activation_gradients = []
        self.weights_gradients = []
        self.biases_gradients = []

        for i in range(len(layers)-1):
            if self.weight_init == 'xavier':
                std = np.sqrt(2/(layers[i]*layers[i+1]))
                self.weights.append(np.random.normal(0,std,(layers[i],layers[i+1])))
                self.biases.append(np.random.normal(0,std,(layers[i+1])))
            else:
                self.weights.append(np.random.normal(0,0.5,(layers[i],layers[i+1])))
                self.biases.append(np.random.normal(0,0.5,(layers[i+1])))
            self.activations.append(np.zeros(layers[i]))
            self.activation_gradients.append(np.zeros(layers[i+1]))
            self.weights_gradients.append(np.zeros((layers[i],layers[i+1])))
            self.biases_gradients.append(np.zeros(layers[i+1]))

        self.activations.append(np.zeros(layers[-1]))
        
        if optimizer == 'adam':
            self.adam(self.x_train,self.y_train)
        elif optimizer == 'sgd':
            self.sgd(self.x_train,self.y_train)
        elif optimizer == 'momentum':
            self.momentum(self.x_train,self.y_train)
        elif optimizer == 'nesterov':
            self.nesterov(self.x_train,self.y_train)
        elif optimizer == 'nadam':
            self.nadam(self.x_train,self.y_train)
        elif optimizer == 'rmsprop':
            self.rmsprop(self.x_train,self.y_train)


    def sigmoid(self,activations):
        res = []
        for z in activations:
            if z>40:
                res.append(1.0)
            elif z<-40:
                res.append(0.0)
            else:
                res.append(1/(1+np.exp(-z)))
        return np.asarray(res)

    def tanh(self,activations):
        res = []
        for z in activations:
            if z>20:
                res.append(1.0)
            elif z<-20:
                res.append(-1.0)
            else:
                res.append((np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z)))
        return np.asarray(res)

    def relu(self,activations):
        res = []
        for i in activations:
            if i<= 0:
                res.append(0)
            else:
                res.append(i)
        return np.asarray(res)

    def softmax(self,activations):
        tot = 0
        for z in activations:
            tot += np.exp(z)
        return np.asarray([np.exp(z)/tot for z in activations])

    def forward_propagation(self,x,y,weights,biases):
        self.activations[0] = x
        n = len(self.layers)
        for i in range(n-2):
            if self.activation_func == "sigmoid":
                self.activations[i+1] = self.sigmoid(np.matmul(weights[i].T,self.activations[i])+biases[i])
            elif self.activation_func == "tanh":
                self.activations[i+1] = self.tanh(np.matmul(weights[i].T,self.activations[i])+biases[i])
            elif self.activation_func == "relu":
                self.activations[i+1] = self.relu(np.matmul(weights[i].T,self.activations[i])+biases[i])

        self.activations[n-1] = self.softmax(np.matmul(weights[n-2].T,self.activations[n-2])+biases[n-2])        
        return -(np.log2(self.activations[-1][y])) #Return cross entropy loss for single data point.


    def grad_w(self,i):
        return np.matmul(self.activations[i].reshape((-1,1)),self.activation_gradients[i].reshape((1,-1)))


    def grad_b(self,i):
        return self.activation_gradients[i]


    def backward_propagation(self,x,y,weights,biases):
        y_onehot = np.zeros(self.output_dim)
        y_onehot[y] = 1
        n = len(self.layers)

        self.activation_gradients[-1] =  -1*(y_onehot - self.activations[-1])
        for i in range(n-2,-1,-1):
            self.weights_gradients[i] += self.grad_w(i)
            self.biases_gradients[i] += self.grad_b(i)
            if i!=0:
                value = np.matmul(weights[i],self.activation_gradients[i])
                if self.activation_func == "sigmoid":
                    self.activation_gradients[i-1] = value * self.activations[i] * (1-self.activations[i])
                elif self.activation_func == "tanh":
                    self.activation_gradients[i-1] = value * (1-np.square(self.activations[i]))
                elif self.activation_func == "relu":
                    res = []
                    for k in self.activations[i]:
                        if k>0: res.append(1.0)
                        else: res.append(0.0)
                    res = np.asarray(res)
                    self.activation_gradients[i-1] = value * res


    def gradient_descent(self,x_train,y_train):
        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]
            
            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)

                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        self.weights[j] -= self.learning_rate * self.weights_gradients[j]
                        self.biases[j] -= self.learning_rate * self.biases_gradients[j]
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                index += 1 

            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',round(self.calculate_accuracy(x_train,y_train),3))


    def sgd(self,x_train,y_train):
        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)

                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        self.weights[j] -= learning_rate * self.weights_gradients[j]
                        self.biases[j] -= learning_rate * self.biases_gradients[j]
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]

            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)

    
    def momentum(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        v_w = (self.decay_rate * prev_gradients_w[j] + self.learning_rate * self.weights_gradients[j])
                        v_b = (self.decay_rate * prev_gradients_b[j] + self.learning_rate * self.biases_gradients[j])
                        self.weights[j] -= v_w
                        self.biases[j] -= v_b
                        prev_gradients_w[j] = v_w
                        prev_gradients_b[j] = v_b
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def nesterov(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            weights = [self.weights[j] -  self.decay_rate * prev_gradients_w[j] for j in range(len(self.weights))]
            biases = [self.biases[j] -  self.decay_rate * prev_gradients_b[j] for j in range(len(self.biases))]

            self.weights_gradients = [0*j for j in (self.weights_gradients)]
            self.biases_gradients = [0*j for j in (self.biases_gradients)]
            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,weights,biases)
                self.backward_propagation(x,y,weights,biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        prev_gradients_w[j] = self.decay_rate * prev_gradients_w[j] + self.learning_rate*self.weights_gradients[j] 
                                           
                        prev_gradients_b[j] = self.decay_rate * prev_gradients_b[j] + self.learning_rate*self.biases_gradients[j] 
                                        
                        self.weights[j] -= prev_gradients_w[j]
                        self.biases[j] -= prev_gradients_b[j]

                    weights = [self.weights[j] -  self.decay_rate * prev_gradients_w[j] for j in range(len(self.weights))]
                    biases = [self.biases[j] -  self.decay_rate * prev_gradients_b[j] for j in range(len(self.biases))]

                    self.weights_gradients = [0*j for j in (self.weights_gradients)]
                    self.biases_gradients = [0*j for j in (self.biases_gradients)]
                    
                index += 1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def rmsprop(self,x_train,y_train):
        prev_gradients_w = [0*i for i in (self.weights_gradients)]
        prev_gradients_b = [0*i for i in (self.biases_gradients)]
        eps = 1e-2

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0

            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        v_w = (self.beta * prev_gradients_w[j] + (1-self.beta) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta * prev_gradients_b[j] + (1-self.beta) * np.square(self.biases_gradients[j]))
                        self.weights[j] -= self.learning_rate * (self.weights_gradients[j] /(np.sqrt(v_w + eps)))
                        self.biases[j] -= self.learning_rate * (self.biases_gradients[j] /(np.sqrt(v_b + eps)))
                        prev_gradients_w[j] = v_w
                        prev_gradients_b[j] = v_b
                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def adam(self,x_train,y_train):
        m_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        m_prev_gradients_b = [0*i for i in (self.biases_gradients)]
        v_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        v_prev_gradients_b = [0*i for i in (self.biases_gradients)]

        iter = 1

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0
            eps = 1e-2
            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):
                        m_w = (self.beta1 * m_prev_gradients_w[j] + (1-self.beta1) * self.weights_gradients[j])
                        m_b = (self.beta1 * m_prev_gradients_b[j] + (1-self.beta1) * self.biases_gradients[j])
                        v_w = (self.beta2 * v_prev_gradients_w[j] + (1-self.beta2) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta2 * v_prev_gradients_b[j] + (1-self.beta2) * np.square(self.biases_gradients[j]))

                        m_hat_w = (m_w)/(1-(self.beta1)**iter) 
                        m_hat_b = (m_b)/(1-(self.beta1)**iter) 

                        v_hat_w = (v_w)/(1-(self.beta2)**iter) 
                        v_hat_b = (v_b)/(1-(self.beta2)**iter)

                        self.weights[j] -= self.learning_rate * (m_hat_w/(np.sqrt(v_hat_w + eps)))
                        self.biases[j] -= self.learning_rate * (m_hat_b/(np.sqrt(v_hat_b + eps)))

                        m_prev_gradients_w[j] = m_w
                        m_prev_gradients_b[j] = m_b
                        v_prev_gradients_w[j] = v_w
                        v_prev_gradients_b[j] = v_b

                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                    iter += 1

                index +=1
            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)

    def nadam(self,x_train,y_train):
        m_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        m_prev_gradients_b = [0*i for i in (self.biases_gradients)]
        v_prev_gradients_w = [0*i for i in (self.weights_gradients)]
        v_prev_gradients_b = [0*i for i in (self.biases_gradients)]

        iter = 1

        for i in range(self.epochs):
            print('Epoch---',i+1,end=" ")
            loss = 0
            val_loss=0
            eps = 1e-2
            self.weights_gradients = [0*i for i in (self.weights_gradients)]
            self.biases_gradients = [0*i for i in (self.biases_gradients)]

            index = 1
            for x,y in zip(x_train,y_train):
                x = x.ravel()
                loss += self.forward_propagation(x,y,self.weights,self.biases)
                self.backward_propagation(x,y,self.weights,self.biases)
                if index % self.batch == 0 or index == x_train.shape[0]:
                    for j in range(len(self.weights)):

                        m_w = (self.beta1 * m_prev_gradients_w[j] + (1-self.beta1) * self.weights_gradients[j])
                        m_b = (self.beta1 * m_prev_gradients_b[j] + (1-self.beta1) * self.biases_gradients[j])
                        v_w = (self.beta2 * v_prev_gradients_w[j] + (1-self.beta2) * np.square(self.weights_gradients[j]))
                        v_b = (self.beta2 * v_prev_gradients_b[j] + (1-self.beta2) * np.square(self.biases_gradients[j]))

                        m_hat_w = (m_w)/(1-(self.beta1)**iter) 
                        m_hat_b = (m_b)/(1-(self.beta1)**iter) 

                        v_hat_w = (v_w)/(1-(self.beta2)**iter) 
                        v_hat_b = (v_b)/(1-(self.beta2)**iter)

                        m_dash_w = self.beta1 * m_hat_w + (1-self.beta1) * self.weights_gradients[j]
                        m_dash_b = self.beta1 * m_hat_b + (1-self.beta1) * self.biases_gradients[j]

                        self.weights[j] -= self.learning_rate * (m_dash_w/(np.sqrt(v_hat_w + eps)))
                        self.biases[j] -= self.learning_rate * (m_dash_b/(np.sqrt(v_hat_b + eps)))

                        m_prev_gradients_w[j] = m_w
                        m_prev_gradients_b[j] = m_b
                        v_prev_gradients_w[j] = v_w
                        v_prev_gradients_b[j] = v_b

                    self.weights_gradients = [0*i for i in (self.weights_gradients)]
                    self.biases_gradients = [0*i for i in (self.biases_gradients)]
                    iter += 1

                index +=1

            for x,y in zip(self.x_cv,self.y_cv):
               x=x.ravel()
               val_loss+=self.forward_propagation(x,y,self.weights,self.biases)

            acc=round(self.calculate_accuracy(x_train,y_train),3)
            val_acc=round(self.calculate_accuracy(self.x_cv,self.y_cv),3)
            wandb.log({'train_loss':loss/x_train.shape[0],'train_accuracy':acc,'val_loss':val_loss/self.x_cv.shape[0],'val_accuarcy':val_acc})
            print('  loss = ',loss/x_train.shape[0],'  accuracy = ',acc,'   validation loss= ',val_loss/self.x_cv.shape[0],'  validation accuaracy= ',val_acc)


    def calculate_accuracy(self,X,Y):
        count = 0
        for i in range(len(X)):
            if self.predict(X[i]) == Y[i]:
                count+=1
        return count/len(X)


    def predict(self,x):
        x = x.ravel()
        self.activations[0] = x
        n = len(self.layers)
        for i in range(n-2):
            if self.activation_func == "sigmoid":
                self.activations[i+1] = self.sigmoid(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])
            elif self.activation_func == "tanh":
                self.activations[i+1] = self.tanh(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])
            elif self.activation_func == "relu":
                self.activations[i+1] = self.relu(np.matmul(self.weights[i].T,self.activations[i])+self.biases[i])

        self.activations[n-1] = self.softmax(np.matmul(self.weights[n-2].T,self.activations[n-2])+self.biases[n-2])

        return np.argmax(self.activations[-1])

In [ ]:
sweep_config={
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters':{
        'epochs':{
            'values':[3,5,7]
        },
        'batch_size':{
            'values':[32,64,128]
        },
        'hidden_layers':{
            'values':[1,2,3]
        },
        'hidden_layers_size':{
            'values':[16,32,64]
        },
        'learning_rate':{
            'values':[5e-3,2e-3,6e-3,5e-4]
        },
        'weight_decay':{
            'values':[0.1,0.0,0.8,0.9]
        },
        'optimizer':{
            'values':['sgd','momentum','nesterov','adam','rmsprop','nadam']
        },
        'activation':{
            'values':['sigmoid','tanh','relu']
        },
        'weight_init':{
            'values':['random','xavier']
        }
    }
}

In [ ]:
!pip install --upgrade wandb
import wandb
!wandb login 3c967c63b099a3b2acd600aa30008e7de1ea6498

In [ ]:
sweep_id = wandb.sweep(sweep_config,project="fashion_mnist", entity="adi00510")

Create sweep with ID: d4znbkb2
Sweep URL: https://wandb.ai/adi00510/fashion_mnist/sweeps/d4znbkb2


In [ ]:
def train():
    config_defaults={
      'epochs':5,
      'batch_size':16,
      'learning_rate':1e-3,
      'activation':'relu',
      'optimizer':'adam',
      'hidden_layers_size':32,
      'hidden_layers':3,
      'weight_init':'xavier' }
    
    
    
    wandb.init(config=config_defaults)
    config=wandb.config

    Neural_network(x_train,y_train,len(x_train[0].ravel()),config.hidden_layers_size,config.hidden_layers,max(y_train)+1,
                       config.batch_size,config.epochs,config.activation,
                 config.learning_rate,config.weight_decay,0.9,0.9,0.99
                        ,config.optimizer,config.weight_init)
    

In [ ]:
wandb.agent(sweep_id,train)

wandb: Agent Starting Run: l9azpgwj with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_layers: 2
wandb: 	hidden_layers_size: 64
wandb: 	learning_rate: 0.005
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0
wandb: 	weight_init: xavier
wandb: Currently logged in as: adi00510 (use `wandb login --relogin` to force relogin)


Epoch--- 1   loss =  0.8558125452399675   accuracy =  0.839    validation loss=  0.6610315860440723   validation accuaracy=  0.829
Epoch--- 2   loss =  0.6316359824747697   accuracy =  0.855    validation loss=  0.6021784636575078   validation accuaracy=  0.848
Epoch--- 3   loss =  0.5814077742344486   accuracy =  0.857    validation loss=  0.5917703408837156   validation accuaracy=  0.85
Epoch--- 4   loss =  0.5539085294280416   accuracy =  0.862    validation loss=  0.5956601487040619   validation accuaracy=  0.854
Epoch--- 5   loss =  0.5269468574386187   accuracy =  0.868    validation loss=  0.563540264770224   validation accuaracy=  0.858


train_loss,0.52695
train_accuracy,0.868
val_loss,0.56354
val_accuarcy,0.858
_runtime,1284
_timestamp,1615376455
_step,4


train_loss,█▃▂▂▁
train_accuracy,▁▅▅▇█
val_loss,█▄▃▃▁
val_accuarcy,▁▆▆▇█
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: clbxemky with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	epochs: 7
wandb: 	hidden_layers: 2
wandb: 	hidden_layers_size: 32
wandb: 	learning_rate: 0.005
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0
wandb: 	weight_init: random


...

wandb: Agent Starting Run: q22sdpir with config:
wandb: 	activation: relu
wandb: 	batch_size: 128
wandb: 	epochs: 3
wandb: 	hidden_layers: 2
wandb: 	hidden_layers_size: 32
wandb: 	learning_rate: 0.0005
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.8
wandb: 	weight_init: random


Epoch--- 1   loss =  27.224129982981705   accuracy =  0.122    validation loss=  27.25777142096648   validation accuaracy=  0.119
Epoch--- 2   loss =  27.224129982981705   accuracy =  0.122    validation loss=  27.25777142096648   validation accuaracy=  0.119
Epoch--- 3   loss =  27.224129982981705   accuracy =  0.122    validation loss=  27.25777142096648   validation accuaracy=  0.119


train_loss,27.22413
train_accuracy,0.122
val_loss,27.25777
val_accuarcy,0.119
_runtime,117
_timestamp,1615394529
_step,2


train_loss,▁▁▁
train_accuracy,▁▁▁
val_loss,▁▁▁
val_accuarcy,▁▁▁
_runtime,▁▄█
_timestamp,▁▄█
_step,▁▅█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
